In [1]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('clean_data.csv')
df_imdb = pd.read_csv('clean_imdb.csv')

In [3]:
np.random.seed(42)
pdf_main = df_main.reindex(np.random.permutation(df_main.index))
pdf_imdb = df_imdb.reindex(np.random.permutation(df_imdb.index))

In [4]:
X_train, X_test, y_train, y_test = pdf_imdb['clean_text'], pdf_main['clean_text'], pdf_imdb['class'], pdf_main['class']

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [7]:
import os
embeddings_index = {}
f = open('glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [8]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [9]:
EMBEDDING_DIM = 200
MAX_SEQUENCE_LENGTH = 200
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [10]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

W0727 17:23:28.114816 140044498167616 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [11]:
pad_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
pad_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128, dropout=.25, recurrent_dropout=.25, return_sequences=True))
model.add(LSTM(128, dropout=.25, recurrent_dropout=.25, return_sequences=True))
model.add(LSTM(128, dropout=.25, recurrent_dropout=.25))
model.add(Dropout(.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

W0727 17:23:29.693296 140044498167616 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 17:23:29.717115 140044498167616 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 17:23:29.741252 140044498167616 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0727 17:23:29.744003 140044498167616 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please us

In [13]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [14]:
check = ModelCheckpoint('rnn200.hdf5', verbose=1, save_best_only=True)
es = EarlyStopping(verbose=1, patience=3)

In [15]:
np.random.seed(42)
model.fit(pad_train, y_train, epochs=15, batch_size=128, validation_data=(pad_test, y_test), callbacks=[check, es])

Train on 50000 samples, validate on 2000 samples
Epoch 1/15
50000/50000 [==============================] - 1877s 38ms/step - loss: 0.4457 - acc: 0.7921 - val_loss: 0.3401 - val_acc: 0.8555

Epoch 00001: val_loss improved from inf to 0.34014, saving model to rnn200.hdf5
Epoch 2/15
50000/50000 [==============================] - 7242s 145ms/step - loss: 0.2663 - acc: 0.8955 - val_loss: 0.3192 - val_acc: 0.8640

Epoch 00002: val_loss improved from 0.34014 to 0.31924, saving model to rnn200.hdf5
Epoch 3/15
50000/50000 [==============================] - 1996s 40ms/step - loss: 0.1888 - acc: 0.9309 - val_loss: 0.3182 - val_acc: 0.8665

Epoch 00003: val_loss improved from 0.31924 to 0.31816, saving model to rnn200.hdf5
Epoch 4/15
50000/50000 [==============================] - 2282s 46ms/step - loss: 0.1298 - acc: 0.9541 - val_loss: 0.4080 - val_acc: 0.8555

Epoch 00004: val_loss did not improve from 0.31816
Epoch 5/15
50000/50000 [==============================] - 2155s 43ms/step - loss: 0.089

In [19]:
model.load_weights('rnn200.hdf5')

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, model.predict_classes(pad_test))

0.8665